In [6]:
import sys
import sklearn
import numpy as np
import os
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [7]:
import quandl as q 

In [8]:
q.ApiConfig.api_key = '11sy5SMsFERx_YypjsMi'

In [9]:
data = q.get('BITFINEX/BTCUSD') 

In [10]:
data

,High,Low,Mid,Last,Bid,Ask,Volume
Date,,,,,,,
2014-04-15,513.9000,452.00,504.23500,505.000000,503.5000,504.97,21013.584774
2014-04-16,547.0000,495.00,537.50000,538.000000,537.0000,538.00,29633.358705
2014-04-17,538.5000,486.10,507.02000,508.000000,506.0400,508.00,20709.783819
2014-04-18,509.0000,474.25,483.77000,482.750000,482.7500,484.79,10458.045243
2014-04-19,513.9899,473.83,505.01065,507.499900,502.5313,507.49,8963.618369
...,...,...,...,...,...,...,...
2021-10-27,61450.0000,57950.00,58518.00000,58517.000000,58517.0000,58519.00,7184.592265
2021-10-28,62444.0000,57606.00,60599.50000,60600.000000,60599.0000,60600.00,7949.438417
2021-10-29,62980.0000,60188.00,62292.50000,62295.000000,62292.0000,62293.00,4041.750450
